In [ ]:
# Paso 1.
# Preprocese los datos.
# - Cree la columna 'Age' a partir de la columna 'Year'.
#   Asuma que el año actual es 2021.
# - Elimine las columnas 'Year' y 'Car_Name'.

In [43]:
import pandas as pd

#Cargar los archivos
train_data = pd.read_csv("../files/input/train_data.csv.zip", compression ="zip",)
#train_data.head()
test_data = pd.read_csv("../files/input/test_data.csv.zip", compression="zip",)
#test_data.head()

#Procesamiento de los datos
def transformar_data(data):
    data["Age"] = 2021 - data["Year"]
    data = data.drop(["Year","Car_Name"], axis=1) # axis=1 indica que se está eliminando columnas
    return data

#Aplicar la función
train_data = transformar_data(train_data)
test_data = transformar_data(test_data)

#Revisar resultados
train_data.head()
test_data.head()


,Selling_Price,Present_Price,Driven_kms,Fuel_Type,Selling_type,Transmission,Owner,Age
0,4.75,9.54,43000,Diesel,Dealer,Manual,0,8
1,7.25,9.85,6900,Petrol,Dealer,Manual,0,4
2,2.85,4.15,5200,Petrol,Dealer,Manual,0,10
3,6.75,8.12,18796,Petrol,Dealer,Manual,0,6
4,6.50,8.61,33429,Diesel,Dealer,Manual,0,6


In [ ]:
# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.

In [44]:
# Crear copias de los datasets originales para trabajar
train_data_copy = train_data.copy()
test_data_copy = test_data.copy()

# Dividir los datos en X (características) e y (variable objetivo)
# Para el dataset de entrenamiento
X_train = train_data_copy.drop(columns=["Present_Price"])  # Todas las columnas excepto la columna objetivo
y_train = train_data_copy["Present_Price"]  # Solo la columna objetivo

# Para el dataset de prueba
X_test = test_data_copy.drop(columns=["Present_Price"])  # Todas las columnas excepto la columna objetivo
y_test = test_data_copy["Present_Price"]  # Solo la columna objetivo

print("Datos divididos:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")


Datos divididos:
X_train: (211, 7), y_train: (211,)
X_test: (90, 7), y_test: (90,)


In [ ]:
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las variables numéricas al intervalo [0, 1].
# - Selecciona las K mejores entradas.
# - Ajusta un modelo de regresion lineal.

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression

# Paso 1: Identificar variables categóricas y numéricas
categorical_columns = ["Fuel_Type", "Selling_type", "Transmission"]
numeric_columns = [col for col in X_train.columns if col not in categorical_columns]

# Paso 2: Transformaciones para cada tipo de variable
categorical_transformer = OneHotEncoder(handle_unknown='ignore')  # One-hot encoding
numerical_transformer = MinMaxScaler()  # Escalado al rango [0, 1]

# Paso 3: Crear un preprocesador para transformar variables categóricas y numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_columns),  # Transformar numéricas
        ('cat', categorical_transformer, categorical_columns)  # Transformar categóricas
    ]
)

# Paso 4: Construir el pipeline completo
pipeline = Pipeline([
    ('preprocessor', preprocessor),  # Transformaciones iniciales
    ('feature_selection', SelectKBest(score_func=f_regression)),  # Seleccionar las K mejores características
    ('model', LinearRegression())  # Modelo de regresión lineal
])

print("Pipeline creado con éxito:")

Pipeline creado con éxito:


In [ ]:
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use el error medio absoluto
# para medir el desempeño modelo.

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error

# Definir los hiperparámetros a optimizar
param_grid = {
    'feature_selection__k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,12],  # Diferentes valores para el número de características a seleccionar
    'model__fit_intercept': [True,False],  # Ajustar o no el intercepto
}

# Crear un objeto para medir el error medio absoluto
#mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

# Configurar la búsqueda de hiperparámetros con validación cruzada
grid_search = GridSearchCV(
    estimator=pipeline,                # El pipeline a optimizar
    param_grid=param_grid,             # Espacio de búsqueda de hiperparámetros
    scoring='neg_mean_absolute_error',                # Métrica de desempeño (MAE)
    cv=10,                             # Número de particiones para validación cruzada
    n_jobs=-1,                         # Usar todos los núcleos disponibles
    verbose=1                          # Mostrar progreso detallado
)

# Ajustar la búsqueda de hiperparámetros a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostrar los mejores hiperparámetros y el mejor desempeño
print("Mejores hiperparámetros:", grid_search.best_params_)
print(f"Mejor MAE obtenido: {-grid_search.best_score_:.4f}")  # Cambiar a positivo

# Actualizar el pipeline con los mejores parámetros encontrados
best_pipeline = grid_search.best_estimator_


Fitting 10 folds for each of 24 candidates, totalling 240 fits
Mejores hiperparámetros: {'feature_selection__k': 11, 'model__fit_intercept': False}
Mejor MAE obtenido: 1.7810


In [ ]:
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.

In [52]:
import os
import pickle
import gzip

# Definir la ruta del directorio y archivo
dir_path = '../files/models'
model_path = '../files/models/model.pkl.gz'

# Crear el directorio si no existe
os.makedirs(dir_path, exist_ok=True)

# Guardar el objeto `grid_search.best_estimator_` comprimido con gzip
with gzip.open(model_path, 'wb') as f:
    pickle.dump(grid_search, f)

print(f"Modelo guardado exitosamente en {model_path}")

Modelo guardado exitosamente en ../files/models/model.pkl.gz


In [ ]:
# Paso 6.
# Calcule las metricas r2, error cuadratico medio, y error absoluto medio
# para los conjuntos de entrenamiento y prueba. Guardelas en el archivo
# files/output/metrics.json. Cada fila del archivo es un diccionario con
# las metricas de un modelo. Este diccionario tiene un campo para indicar
# si es el conjunto de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
# {'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}

In [54]:
import json
import os
from sklearn.metrics import mean_squared_error, median_absolute_error, r2_score

# Asegurarnos de usar el mejor modelo encontrado
final_model = grid_search.best_estimator_

# Realizar predicciones
y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)

# Calcular métricas para el conjunto de entrenamiento
train_metrics = {
    "type": "metrics",
    'dataset': 'train',
    'r2': r2_score(y_train, y_train_pred),
    'mse': mean_squared_error(y_train, y_train_pred),
    'mad': median_absolute_error(y_train, y_train_pred)
}

# Calcular métricas para el conjunto de prueba
test_metrics = {
    "type": "metrics",
    'dataset': 'test',
    'r2': r2_score(y_test, y_test_pred),
    'mse': mean_squared_error(y_test, y_test_pred),
    'mad': median_absolute_error(y_test, y_test_pred)
}

# Definir la ruta del archivo de salida
output_dir = "../files/output"
output_path = os.path.join(output_dir, "metrics.json")

# Crear las carpetas necesarias si no existen
os.makedirs(output_dir, exist_ok=True)

# Guardar las métricas en un archivo JSON
with open(output_path, 'w') as f:
    json.dump(train_metrics, f)  # Guardar las métricas de entrenamiento
    f.write("\n")  # Escribir una nueva línea
    json.dump(test_metrics, f)  # Guardar las métricas de prueba

print(f"Métricas guardadas exitosamente en {output_path}")


Métricas guardadas exitosamente en ../files/output\metrics.json
